In [22]:
import requests
from bs4 import BeautifulSoup
import re

# URL da página que queremos extrair dados
url = 'https://www.investing.com/commodities/brent-oil'

# Enviar uma requisição GET para obter o conteúdo da página
response = requests.get(url)

# Verificar se a requisição foi bem sucedida (código 200 indica sucesso)
if response.status_code == 200:
    # Utilizar BeautifulSoup para fazer o parsing do conteúdo HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Contador para controlar as três primeiras ocorrências
    count = 0
    
    # Iterar sobre os elementos que contêm números e texto associado
    for element in soup.find_all(text=re.compile(r'\d+')):
        if element.parent.name in ['p', 'div', 'span']:  # Exemplo de tags onde os números podem estar
            text = element.parent.get_text().strip()
            number = element.strip()
            print(f'Texto: {text} - Número: {number}')
            count += 1
            if count >= 3:  # Parar após imprimir as três primeiras ocorrências
                break
    
    print(f'Total de pares encontrados: {count}')
else:
    print(f'Erro ao acessar a página: {response.status_code}')


Texto: Get 100% ad-free experience - Número: Get 100% ad-free experience
Texto: 86.86 - Número: 86.86
Texto: -0.57 - Número: -0.57
Total de pares encontrados: 3


/tmp/ipykernel_346/3951380360.py:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for element in soup.find_all(text=re.compile(r'\d+')):


In [23]:
import socket

def check_port(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.bind(("localhost", port))
        return True
    except Exception as e:
        return False
    finally:
        sock.close()

def find_available_port(start_port, end_port):
    for port in range(start_port, end_port + 1):
        if check_port(port):
            return port
    return None

# Exemplo de uso: encontrar uma porta disponível no intervalo de 5000 a 6000
start_port = 5000
end_port = 6000
available_port = find_available_port(start_port, end_port)

if available_port:
    print(f"Porta disponível encontrada: {available_port}")
else:
    print("Nenhuma porta disponível encontrada no intervalo.")


Porta disponível encontrada: 5000


In [24]:
from flask import Flask, jsonify
import requests
from bs4 import BeautifulSoup
import re

app = Flask(__name__)

# URL da página que queremos extrair dados
url = 'https://www.investing.com/commodities/brent-oil'

# Função para obter o preço do Brent Oil
def get_brent_oil_price():
    # Enviar uma requisição GET para obter o conteúdo da página
    response = requests.get(url)
    
    # Verificar se a requisição foi bem sucedida (código 200 indica sucesso)
    if response.status_code == 200:
        # Utilizar BeautifulSoup para fazer o parsing do conteúdo HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Procurar o elemento que contém o preço
        price_element = soup.find('span', {'id': 'last_last'})
        
        if price_element:
            price = price_element.text.strip()
            return price
        else:
            return 'Preço não encontrado'
    else:
        return f'Erro ao acessar a página: {response.status_code}'

# Rota para obter o preço do Brent Oil
@app.route('/brent-oil-price', methods=['GET'])
def brent_oil_price():
    price = get_brent_oil_price()
    return jsonify({'price': price})

# Executar o servidor Flask no Kaggle
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('127.0.0.1', 5000, app)
